In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8,application_1622616251750_0015,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%configure -f
{ "conf":{    
          "spark.databricks.hive.metastore.glueCatalog.enabled" : "true",
          "spark.jars.packages": "io.delta:delta-core_2.12:0.8.0",
          "spark.sql.extensions": "io.delta.sql.DeltaSparkSessionExtension",
          "spark.sql.catalog.spark_catalog": "org.apache.spark.sql.delta.catalog.DeltaCatalog",
          "spark.sql.warehouse.dir": "s3://aws-poc-serverless-analytics/delta_lake_demo/clean/"
    
         }
}


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
9,application_1622616251750_0016,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
9,application_1622616251750_0016,pyspark,idle,Link,Link,✔


In [24]:
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
clean_table = "user_cln"
database = "delta_lake_demo"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark.sql(f"DROP TABLE IF EXISTS {clean_table}")
spark.sql(f"DROP TABLE IF EXISTS {clean_table}_external")
#spark.sql(f"DROP DATABASE IF EXISTS {database}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [5]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS {database}")
spark.sql(f"USE {database}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [6]:
spark.sql(f"""
            CREATE TABLE user_cln(
            user_id int,
            sign_in_cnt int,
            current_sign_in_utc_ts timestamp,
            last_sign_in_utc_ts timestamp,
            failed_attempts int,
            unlock_token string,
            locked_utc_ts timestamp,
            merchant_id int,
            create_utc_ts timestamp,
            update_utc_ts timestamp,
            invitation_token string,
            invitation_create_utc_ts timestamp,
            invitation_sent_utc_ts timestamp,
            invitation_accept_utc_ts timestamp,
            invitation_limit_num int,
            invited_by_id int,
            invited_by_type string,
            invitations_cnt int,
            uuid string,
            utm_src string,
            utm_campaign string,
            utm_medium string,
            utm_term string,
            utm_content string,
            tutorial_completed_create_listing_flag boolean,
            tutorial_completed_enable_inventory_sync_flag boolean,
            tutorial_completed_fulfill_order_flag boolean,
            time_zone string,
            cln_insert_utc_ts timestamp,
            cln_update_utc_ts timestamp)
            USING DELTA
            PARTITIONED BY (sign_in_cnt)
            LOCATION 's3://aws-poc-serverless-analytics/delta_lake_demo/clean/delta_lake_demo.db/user_cln/'
            """)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [7]:
spark.sql(f"""GENERATE symlink_format_manifest FOR TABLE {clean_table}""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [8]:
spark.sql(f""" ALTER TABLE {clean_table} SET TBLPROPERTIES(delta.compatibility.symlinkFormatManifest.enabled=true)""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [9]:
spark.sql(f""" CREATE EXTERNAL TABLE IF NOT EXISTS {clean_table}_external(
                user_id int,
                current_sign_in_utc_ts timestamp,
                last_sign_in_utc_ts timestamp,
                failed_attempts int,
                unlock_token string,
                locked_utc_ts timestamp,
                merchant_id int,
                create_utc_ts timestamp,
                update_utc_ts timestamp,
                invitation_token string,
                invitation_create_utc_ts timestamp,
                invitation_sent_utc_ts timestamp,
                invitation_accept_utc_ts timestamp,
                invitation_limit_num int,
                invited_by_id int,
                invited_by_type string,
                invitations_cnt int,
                uuid string,
                utm_src string,
                utm_campaign string,
                utm_medium string,
                utm_term string,
                utm_content string,
                tutorial_completed_create_listing_flag boolean,
                tutorial_completed_enable_inventory_sync_flag boolean,
                tutorial_completed_fulfill_order_flag boolean,
                time_zone string,
                cln_insert_utc_ts timestamp,
                cln_update_utc_ts timestamp)
                PARTITIONED BY (sign_in_cnt int)
                ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
                STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
                OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
                LOCATION 's3://aws-poc-serverless-analytics/delta_lake_demo/clean/delta_lake_demo.db/user_cln/_symlink_format_manifest/' 
                """)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [10]:
spark.stop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from http://localhost:8998/sessions/9/statements/8 with error payload: {"msg":"requirement failed: Session isn't active."}


In [92]:
spark.sql(f"""MSCK REPAIR TABLE {clean_table}_external""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

###  SCD 2 

In [15]:
spark.sql(f"""
            ALTER TABLE {clean_table} ADD columns (active_flag string)
          """)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [18]:
spark.sql(f"""
            UPDATE {clean_table} SET active_flag = 'true'
            """)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [20]:
spark.sql(f"""
            ALTER TABLE {clean_table}_external ADD columns (active_flag string)
          """)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [19]:
spark.sql(f"""SELECT * FROM {clean_table}""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------------+------------+-----------+--------------------+--------------------+-----------+
|    id|        company_name|   telephone|      state|          created_at|          updated_at|active_flag|
+------+--------------------+------------+-----------+--------------------+--------------------+-----------+
| 44083|Quick Draw Outdoo...|  3182596422|        abc|2019-02-04 16:57:...|2021-02-10 02:07:...|       true|
| 44082|Slow Draw Outdoor...|  3182596422|provisioned|2019-02-04 16:57:...|2021-02-11 02:07:...|       true|
|156237|          Washer Fan|000-000-0000|provisioned|2020-10-26 20:10:...|2020-10-26 20:53:...|       true|
|177863|  Bellelyse Boutique|000-000-0000|  suspended|2020-12-31 04:56:...|2021-02-19 16:53:...|       true|
| 50742|Thenaricalicollec...|  5049394269|  suspended|2019-03-22 03:57:...|2019-05-15 00:54:...|       true|
|176321|   CLICK MY CART LTD|  6479863690|  suspended|2020-12-23 06:20:...|2020-12-24 04:53:...|       true|
|144524|     Rachid

In [78]:
spark.sql(f"""MSCK REPAIR TABLE  {clean_table}_external""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [81]:
spark.stop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…